In [8]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [9]:
req = requests.get('https://vehiculeselectriques.gouv.qc.ca/english/rabais/ve-neuf/vehicules-neufs-admissibles.asp')
soup = BeautifulSoup(req.content, 'html.parser')
with open('Eligible vehicles.html') as Eliveh:
    new_soup = BeautifulSoup(Eliveh,'html.parser')

In [10]:
cars = soup.find_all("div", class_="infoVoiture")
fed_cars = new_soup.find('table')
fed_cars = fed_cars.findAll('tr', role='row')[1:]

In [19]:
fed_cars_df = pd.DataFrame()

for car in fed_cars:
    car_make = car.find('td', class_='sorting_1').text
    car_model = car.find('td', class_='sorting_3').text
    car_model_year = car.find('td', class_='sorting_2').text
    try:
        car_trim = car.find('td', headers='tbl8').text
        car_catagory = car.find('td', headers='tbl9').text
        car_rebate = car.find('td', headers='tbl12').text
        car_eligibility = car.find('td', headers = 'tbl16').text
    except:
        car_trim = ''
        car_catagory = ''
        car_rebate = ''
        car_eligibility = 'Empty'
    fed_cars_df = fed_cars_df.append({'Make':car_make,'Model':car_model, 'Trim':car_trim,'Rebate-QC':car_rebate,'Year-QC':car_model_year,'Catagory':car_catagory,'Eligibility':car_eligibility}, ignore_index=True)

fed_cars_df.loc[fed_cars_df['Eligibility'] == 'Empty']

,Make,Model,Trim,Rebate-QC,Year-QC,Catagory,Eligibility
12,BMW,i3,,,2020,,Empty
27,Ford,*Escape PHEV,,,2022,,Empty
30,Ford,*Escape PHEV,,,2021,,Empty
32,Ford,*Escape PHEV,,,2020,,Empty
41,Hyundai,*Santa Fe PHEV,,,2022,,Empty
42,Hyundai,*Tucson PHEV,,,2022,,Empty
63,Kia,*Sorento PHEV,,,2022,,Empty


In [12]:
cars_df = pd.DataFrame()
makes = ['Audi','BMW','BYD','Chevrolet','Chrysler','Ford','Honda','Hyundai','Jeep','Kia','Lexus','Lincoln','Mazda','Mini','Mitsubishi','Nissan','Polestar','Subaru','Tesla','Toyota','Volkswagen','Volvo']

for car in cars:
    car_catagory = car.findAll('p')[1].text[10:]
    if car_catagory in ['All-electric','Plug-in hybrid']:
        car_string = car.find('h2').text
        car_model_year = car.findAll('p')[0].text[15:-2]
        car_rebate = car.find('div', class_='rabais-achat').text[1:-1]
        for make in makes:
            if make in car_string:
                car_name = car_string[:len(make)]
                car_model = car_string[len(make)+1:]
                continue
        cars_df = cars_df.append({'Make':car_name,'Model':car_model,'Rebate-QC':car_rebate,'Year-QC':car_model_year,'Catagory':car_catagory}, ignore_index=True)

In [24]:
cars_df.loc[cars_df['Make'] == 'BMW']

,Make,Model,Rebate-QC,Year-QC,Catagory
1,BMW,X3 – xDrive30e,$2 500,2020 - 2021 - 2022,Plug-in hybrid
2,BMW,3 Series – 330e or 330e xDrive,$2 500,2021 - 2022,Plug-in hybrid
3,BMW,i3,$7 000,2020 - 2021,All-electric
4,BMW,i3 REx and i3s Rex,$5 000,2020 - 2021,Plug-in hybrid
5,BMW,i3s,$7 000,2020 - 2021,All-electric
6,BMW,i4 eDrive40,$7 000,2022,All-electric


In [25]:
fed_cars_df.loc[fed_cars_df['Make'] == 'BMW']

,Make,Model,Trim,Rebate-QC,Year-QC,Catagory,Eligibility
2,BMW,330e,RWD\n\t\t\txDrive,"$2,500",2023,PHEV,2022-08-05
3,BMW,i4,eDrive35\n\t\t\teDrive40,"$5,000",2023,BEV,2022-08-08
4,BMW,330e,RWD\n\t\t\txDrive,"$2,500",2022,PHEV,2021-07-02
5,BMW,i4,eDrive40,"$5,000",2022,BEV,2022-05-03
6,BMW,X3,xDrive30e,"$2,500",2022,PHEV,2022-04-25
7,BMW,330e,RWD\n\t\t\txDrive,"$2,500",2021,PHEV,2020-04-21
8,BMW,i3,Base\n\t\t\ts,"$5,000",2021,BEV,2020-12-21
9,BMW,i3,Range Extender\n\t\t\ts Range Extender,"$5,000",2021,PHEV,2020-12-21
10,BMW,X3,xDrive30e,"$2,500",2021,PHEV,2022-07-25
11,BMW,i3,Base\n\t\t\ts,"$5,000",2020,BEV,2020-01-16
